# Smash Character Competency Visualizer
Use the controls below and press **Fetch metrics** to pull the latest player data from start.gg. First loads for a new game or state can take up to a minute while caches warm, but once complete the controls update instantly.


In [1]:

import datetime as dt

import pandas as pd
import altair as alt
import ipywidgets as widgets
from IPython.display import display, Markdown

from smashcc.analysis import generate_player_metrics

alt.data_transformers.disable_max_rows()


SMASH_THEME = {
    "config": {
        "background": "#f9fafb",
        "view": {"stroke": "transparent"},
        "axis": {
            "labelColor": "#1f2937",
            "titleColor": "#111827",
            "gridColor": "#e5e7eb",
            "tickColor": "#cbd5f5",
            "labelFontSize": 11,
            "titleFontSize": 12,
            "titleFontWeight": "600",
        },
        "legend": {
            "labelColor": "#1f2937",
            "titleColor": "#111827",
            "symbolType": "circle",
        },
        "title": {"fontSize": 18, "fontWeight": "600"},
        "range": {
            "category": [
                "#2563eb",
                "#f97316",
                "#10b981",
                "#9333ea",
                "#ef4444",
                "#0ea5e9",
                "#facc15",
                "#22c55e",
                "#a855f7",
                "#f59e0b",
                "#64748b",
            ]
        },
        "mark": {"opacity": 0.9},
    }
}

def _smash_theme():
    return SMASH_THEME

alt.themes.register("smash_theme", _smash_theme)
alt.themes.enable("smash_theme")
alt.renderers.set_embed_options(actions=False)


DataTransformerRegistry.enable('default')

In [2]:

METRIC_OPTIONS = {
    "Weighted Win Rate": {"field": "weighted_win_rate", "scale": (0, 1), "format": ".0%"},
    "Win Rate": {"field": "win_rate", "scale": (0, 1), "format": ".0%"},
    "Opponent Strength": {"field": "opponent_strength"},
    "Average Seed Delta": {"field": "avg_seed_delta"},
    "Upset Rate": {"field": "upset_rate", "scale": (0, 1), "format": ".0%"},
    "Average Event Entrants": {"field": "avg_event_entrants"},
    "Events Played": {"field": "events_played"},
    "Sets Played": {"field": "sets_played"},
    "Activity Score": {"field": "activity_score"},
    "Character Usage Rate": {"field": "character_usage_rate", "scale": (0, 1), "format": ".0%"},
}

DEFAULT_X_METRIC = "Opponent Strength"
DEFAULT_Y_METRIC = "Weighted Win Rate"

GAME_OPTIONS = {
    "Super Smash Bros. Ultimate": 1386,
    "Super Smash Bros. Melee": 1,
    "Street Fighter 6": 43868,
    "Tekken 8": 49783,
}

game_dropdown = widgets.Dropdown(
    options=[(label, value) for label, value in GAME_OPTIONS.items()],
    value=1386,
    description="Game",
    layout=widgets.Layout(width="320px"),
)

state_input = widgets.Text(
    value="GA",
    description="State",
    placeholder="GA",
    continuous_update=False,
    layout=widgets.Layout(width="140px"),
)

character_input = widgets.Text(
    value="Marth",
    description="Character",
    placeholder="Marth",
    continuous_update=False,
    layout=widgets.Layout(width="200px"),
)

filter_state_input = widgets.Text(
    value="",
    description="Filter states",
    placeholder="GA,AL",
    continuous_update=False,
    layout=widgets.Layout(width="220px"),
)

months_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=24,
    step=1,
    description="Months",
    continuous_update=False,
    layout=widgets.Layout(width="320px"),
)

min_sets_slider = widgets.IntSlider(
    value=5,
    min=0,
    max=50,
    step=1,
    description="Min sets",
    continuous_update=False,
    layout=widgets.Layout(width="320px"),
)

min_entrants_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=512,
    step=4,
    description="Min entrants",
    continuous_update=False,
    layout=widgets.Layout(width="320px"),
)

max_entrants_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=1024,
    step=4,
    description="Max entrants",
    continuous_update=False,
    layout=widgets.Layout(width="320px"),
)

start_after_picker = widgets.DatePicker(
    description="Start after",
    disabled=False,
    layout=widgets.Layout(width="220px"),
)

x_metric_dropdown = widgets.Dropdown(
    options=list(METRIC_OPTIONS.keys()),
    value=DEFAULT_X_METRIC,
    description="X axis",
    layout=widgets.Layout(width="260px"),
)

y_metric_dropdown = widgets.Dropdown(
    options=list(METRIC_OPTIONS.keys()),
    value=DEFAULT_Y_METRIC,
    description="Y axis",
    layout=widgets.Layout(width="260px"),
)

controls = widgets.VBox([
    widgets.HBox([game_dropdown, state_input, character_input]),
    widgets.HBox([months_slider, filter_state_input, start_after_picker]),
    widgets.HBox([min_sets_slider, min_entrants_slider, max_entrants_slider]),
    widgets.HBox([x_metric_dropdown, y_metric_dropdown]),
])

run_button = widgets.Button(
    description="Fetch metrics",
    button_style="primary",
    icon="refresh",
    layout=widgets.Layout(width="200px", margin="10px 0px 0px 0px"),
)

loading_indicator = widgets.HTML(
    value=(
        "<style>"
        "@keyframes spin {from {transform: rotate(0deg);} to {transform: rotate(360deg);}}"
        ".loader {border: 3px solid #e5e7eb; border-top-color: #2563eb; border-radius: 9999px;"
        " width: 18px; height: 18px; animation: spin 0.9s linear infinite;}"
        "</style>"
        "<div style='display:flex;align-items:center;gap:10px;padding:6px 0;'>"
        "  <div class='loader'></div>"
        "  <span>Fetching data from start.gg… first loads may take up to a minute.</span>"
        "</div>"
    )
)

output = widgets.Output()

_state = {"df": None, "initialized": False, "is_fetching": False}


def _metric_channel(metric_key: str, axis: str):
    meta = METRIC_OPTIONS[metric_key]
    axis_kwargs = {"title": metric_key}
    fmt = meta.get("format")
    if fmt:
        axis_kwargs["format"] = fmt
    axis_obj = alt.Axis(**axis_kwargs)
    channel = alt.X if axis == "x" else alt.Y
    kwargs = {"shorthand": f"{meta['field']}:Q", "axis": axis_obj}
    if meta.get("scale"):
        kwargs["scale"] = alt.Scale(domain=meta["scale"])
    return channel(**kwargs)


def _render(df: pd.DataFrame) -> None:
    filtered = df.copy()
    filtered = filtered[filtered["sets_played"].fillna(0) >= min_sets_slider.value]
    if min_entrants_slider.value:
        filtered = filtered[filtered["avg_event_entrants"].fillna(0) >= min_entrants_slider.value]
    if max_entrants_slider.value:
        filtered = filtered[filtered["avg_event_entrants"].fillna(0) <= max_entrants_slider.value]
    if filter_state_input.value.strip() and "home_state" in filtered:
        allowed = {s.strip().upper() for s in filter_state_input.value.split(",") if s.strip()}
        if allowed:
            home_series = filtered["home_state"].fillna("").astype(str).str.upper()
            filtered = filtered[home_series.isin(allowed)]
    if start_after_picker.value:
        cutoff_dt = dt.datetime.combine(start_after_picker.value, dt.time.min, tzinfo=dt.timezone.utc)
        cutoff_ts = int(cutoff_dt.timestamp())
        filtered = filtered[filtered["latest_event_start"].fillna(0) >= cutoff_ts]

    with output:
        output.clear_output()
        if filtered.empty:
            display(Markdown("No player data available for the current filters."))
            return

        filtered = filtered.copy()
        filtered["latest_event"] = pd.to_datetime(filtered["latest_event_start"], unit="s", errors="coerce")
        filtered["win_rate_pct"] = (pd.to_numeric(filtered["win_rate"], errors="coerce") * 100).round(1)
        filtered["weighted_win_rate_pct"] = (pd.to_numeric(filtered["weighted_win_rate"], errors="coerce") * 100).round(1)
        filtered["upset_rate_pct"] = (pd.to_numeric(filtered["upset_rate"], errors="coerce") * 100).round(1)
        filtered["character_usage_pct"] = (pd.to_numeric(filtered["character_usage_rate"], errors="coerce") * 100).round(1)

        chart = (
            alt.Chart(filtered)
            .mark_circle(size=160, opacity=0.75)
            .encode(
                x=_metric_channel(x_metric_dropdown.value, axis="x"),
                y=_metric_channel(y_metric_dropdown.value, axis="y"),
                color=alt.Color("home_state:N", title="Home State", legend=alt.Legend(orient="bottom")),
                tooltip=[
                    alt.Tooltip("gamer_tag:N", title="Player"),
                    alt.Tooltip("state:N", title="Tournament State"),
                    alt.Tooltip("home_state:N", title="Home State"),
                    alt.Tooltip("events_played:Q", title="Events"),
                    alt.Tooltip("sets_played:Q", title="Sets"),
                    alt.Tooltip("win_rate_pct:Q", title="Win %"),
                    alt.Tooltip("weighted_win_rate_pct:Q", title="Weighted Win %"),
                    alt.Tooltip("avg_event_entrants:Q", title="Avg Entrants"),
                    alt.Tooltip("avg_seed_delta:Q", title="Avg Seed Delta"),
                    alt.Tooltip("upset_rate_pct:Q", title="Upset %"),
                    alt.Tooltip("latest_event:T", title="Latest Event"),
                ],
            )
            .properties(height=420)
            .interactive()
        )

        display(chart)

        sort_field = METRIC_OPTIONS[y_metric_dropdown.value]["field"]
        display_cols = [
            "gamer_tag",
            "state",
            "home_state",
            "events_played",
            "sets_played",
            "win_rate_pct",
            "weighted_win_rate_pct",
            "avg_seed_delta",
            "opponent_strength",
            "avg_event_entrants",
            "upset_rate_pct",
            "character_usage_pct",
            "latest_event",
        ]
        existing_cols = [col for col in display_cols if col in filtered.columns]
        display(filtered.sort_values(sort_field, ascending=False).head(50)[existing_cols])


def refresh(force_fetch: bool = False) -> None:
    if _state["is_fetching"]:
        return
    if force_fetch or _state["df"] is None:
        _state["is_fetching"] = True
        with output:
            output.clear_output()
            display(loading_indicator)
        try:
            df = generate_player_metrics(
                state=state_input.value.strip().upper() or "GA",
                months_back=months_slider.value,
                videogame_id=game_dropdown.value,
                target_character=character_input.value.strip() or "Marth",
            )
        except Exception as exc:
            with output:
                output.clear_output()
                display(Markdown(f"**Error:** {exc}"))
            _state["is_fetching"] = False
            return
        _state["df"] = df
        _state["initialized"] = True
        _state["is_fetching"] = False
        _render(df)
    elif _state["df"] is not None:
        _render(_state["df"])


def _on_fetch_change(change):
    if change.get("name") == "value":
        refresh(force_fetch=True)


def _on_render_change(change):
    if change.get("name") == "value" and _state["df"] is not None:
        refresh(force_fetch=False)


run_button.on_click(lambda _: refresh(force_fetch=True))

game_dropdown.observe(_on_fetch_change, names="value")
months_slider.observe(_on_fetch_change, names="value")

min_sets_slider.observe(_on_render_change, names="value")
min_entrants_slider.observe(_on_render_change, names="value")
max_entrants_slider.observe(_on_render_change, names="value")
start_after_picker.observe(_on_render_change, names="value")
x_metric_dropdown.observe(_on_render_change, names="value")
y_metric_dropdown.observe(_on_render_change, names="value")

filter_state_input.observe(_on_render_change, names="value")
state_input.observe(_on_fetch_change, names="value")
character_input.observe(_on_fetch_change, names="value")


display(controls)
display(run_button)
display(output)

with output:
    display(Markdown("Adjust filters and click **Fetch metrics** to load data."))


Button(button_style='primary', description='Fetch metrics', icon='refresh', layout=Layout(margin='10px 0px 0px…

Output()